## Imports

In [1]:
# Connecting to the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## Importing all dependancies
import pandas as pd
import pickle
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils.data import DataLoader, Dataset
import tqdm
import torch.optim as optim

## Definations

In [13]:
# Pairwise dataset generator
class RelationDataset:

    def __init__(self, edges, true_edges):

    ## edges : dictonary type object containing information of train/validation/test set edges
    ## true_edges : dictonary type object containing information of all the knowledge facts in graphs in form of edges (i.e. union set of train, valid, test edges)


        self.true_edges = true_edges
        self.train_edges = edges
        self.edge_index = edges['edge_index']        # edge_index is edge list tensor for train/validation/test edges
        self.edge_reltype = edges['edge_reltype']    # edge_reltype is edge type tensor for train/validation/test edges
        self.num_nodes = 23763                         # total number of unique nodes involved in train/validation/test edges
        self.num_rels = 7                         # total number of unique relations involved in train/validation/test edges
        self.rel_dict = {}
        self.true_edge_dict = {}    # Dictonary of true triplets where key is tuple (head_node_index, tail_node_index) value is a list [edge1_type_index, edge2_type_index........]


        ## Constructing true edge dict to quickly check if a synthetic triple of a relation type exist among two nodes
        for i in range(self.true_edges['edge_index'].shape[1]) :    # Iterating over every entry in edhe list
          head = self.true_edges['edge_index'][0,i]
          tail = self.true_edges['edge_index'][1,i]
          r = self.true_edges['edge_reltype'][i,0]

          if (head,tail) not in self.true_edge_dict:
            self.true_edge_dict[(head,tail)] =[]

          self.true_edge_dict[(head,tail)].append(r)



    def __len__(self):
        return self.edge_index.size(1)


    def _sample_negative_edge(self, idx):
        '''
        Function to generate a negative sample for a given positive sample
        "idx" : index of edge from train edge list
        '''
        sample = random.uniform(0, 1)
        found = False
        while not found:
          if sample <= 0.5:
            # corrupt the tail entity
            h = self.edge_index[0, idx]
            t = torch.randint(0, self.num_nodes, (1,))
            r = self.edge_reltype[idx,:]
          else :
            # corrupt the head entity
            t = self.edge_index[1, idx]
            h = torch.randint(0, self.num_nodes, (1,))
            r = self.edge_reltype[idx,:]

          # check if the edge is a true edge
          if (h, t) not in self.true_edge_dict:
              found = True
          #elif r not in self.true_edge_dict[(h, t)]:
          #    found = True

        data = [torch.tensor([h,t]), r]
        return data


    def __getitem__(self, idx):
        '''
        Function to generate positive and negative triplet for a given training edge
        "idx" : index of edge from train edge list
        A triplet is a list of two objects : first object is tensor of head and tail node indices, second object is relation type index
        '''
        pos_sample = [self.edge_index[:, idx], self.edge_reltype[idx,:]]
        neg_sample = self._sample_negative_edge(idx)
        return pos_sample, neg_sample

In [14]:
# KG Model Defination
class TransE(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim):
        # The initialization acts like lookup layer where shallow random embeddings are generated for each node and relation
        super(TransE, self).__init__()
        self.entity_embeddings = torch.nn.Parameter(torch.randn(num_entities, embedding_dim))
        self.relation_embeddings = torch.nn.Parameter(torch.randn(num_relations, embedding_dim))

    def forward(self):
      # The function computes l2 norm of every row and devide the node entitity emdedding by their row-wise norms to normalize them
        self.entity_embeddings.data[:-1, :].div_(
            self.entity_embeddings.data[:-1, :].norm(p=2, dim=1, keepdim=True))
        return self.entity_embeddings, self.relation_embeddings

In [17]:
# KG loss defination
def TransE_loss(pos_edges, neg_edges, pos_reltype, neg_reltype, entity_embeddings, relation_embeddings):
      # Select embeddings for both positive and negative samples
      pos_head_embeds = torch.index_select(entity_embeddings, 0, pos_edges[:, 0])
      pos_tail_embeds = torch.index_select(entity_embeddings, 0, pos_edges[:, 1])
      neg_head_embeds = torch.index_select(entity_embeddings, 0, neg_edges[:, 0])
      neg_tail_embeds = torch.index_select(entity_embeddings, 0, neg_edges[:, 1])
      pos_relation_embeds = torch.index_select(relation_embeddings, 0, pos_reltype.squeeze())
      neg_relation_embeds = torch.index_select(relation_embeddings, 0, neg_reltype.squeeze())

      # Calculate the distance score
      d_pos = torch.norm(pos_head_embeds + pos_relation_embeds - pos_tail_embeds, p=1, dim=1)
      d_neg = torch.norm(neg_head_embeds + neg_relation_embeds - neg_tail_embeds, p=1, dim=1)
      ones = torch.ones(d_pos.size(0))

      # margin loss - we want to increase d_neg and decrease d_pos  (loss(x1,x2,y)=max(0,−y∗(x1−x2)+margin))
      margin_loss = torch.nn.MarginRankingLoss(margin=1.)
      loss = margin_loss(d_neg, d_pos, ones)
      #loss = d_pos.mean(dim=0)                             # in computation of loss 1 is gamma value so that true triplet score should be more than 1 that of false triplet score

      return loss

## Workflow Pipeline

### Node & Edge Map dictonary generation


In [3]:
# Generating edge name-index map
edges = ["has-category","co-visited","visited","friend-with","has-aspect","concerned-aspect","situated-at"]
edge_map_dict = {}
for i in range(len(edges)):
  edge_map_dict[edges[i]]= i

# Saving the created edge-index map if required
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_KG_edge_map_dict','wb')as f:
  pickle.dump(edge_map_dict,f)

In [ ]:
# Generating node name-index map
node_map_dict = {}
counter = 0

# Updating the map based on category edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_category_edges_tf_idf.pkl', 'rb') as f:
    category_edges = pickle.load(f)

for i in range(len(category_edges)):
    for item_id in category_edges[i]:
        if item_id not in node_map_dict:
            node_map_dict[item_id] = counter
            counter = counter + 1
print("Number of nodes after adding category edges: ", len(node_map_dict))

# Updating the map based on covisit edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/poi_covisited_edges.pkl', 'rb') as f:
    covisit_edges = pickle.load(f)

for i in range(len(covisit_edges)):
    for item_id in covisit_edges[i]:
        if item_id not in node_map_dict:
            node_map_dict[item_id] = counter
            counter = counter + 1
print("Number of nodes after adding covisit edges: ", len(node_map_dict))



#Updating the map based on geo edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/poi_geocluster_edges.pkl', 'rb') as f:
    geo_edges = pickle.load(f)

for i in range(len(geo_edges)):
    for item_id in geo_edges[i]:
        if item_id not in node_map_dict:
            node_map_dict[item_id] = counter
            counter = counter + 1
print("Number of nodes after adding geo edges: ", len(node_map_dict))


# Updating the map based on visited edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/visited_edges.pkl', 'rb') as f:
    vfive_edges = pickle.load(f)

for i in range(len(vfive_edges)):
    for item_id in vfive_edges[i]:
        if item_id not in node_map_dict:
            node_map_dict[item_id] = counter
            counter = counter + 1
print("Number of nodes after adding visited edges: ", len(node_map_dict))


# Updating the map based on friend edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/friendship_edges.pkl', 'rb') as f:
    friend_edges = pickle.load(f)

for i in range(len(friend_edges)):
    for item_id in friend_edges[i]:
        if item_id not in node_map_dict:
            node_map_dict[item_id] = counter
            counter = counter + 1
print("Number of nodes after adding friend edges: ", len(node_map_dict))


# Updating the map based on review aspect edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/shortlisted_poi_significant_aspect_edges.pkl', 'rb') as f:
    aspect_edges = pickle.load(f)

for i in range(len(aspect_edges)):
    for item_id in aspect_edges[i]:
        if item_id not in node_map_dict:
            node_map_dict[item_id] = counter
            counter = counter + 1
print("Number of nodes after adding category edges: ", len(node_map_dict))

# Creating inverse node map dict
## Inverse node map dict, key:node index, value: node name
inv_node_map_dict = {}
nodes = list(node_map_dict.keys())
for index in range(len(nodes)):
    inv_node_map_dict[index] = nodes[index]

# Saving the created node-index map if required
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_KG_node_map_dict','wb')as f:
  pickle.dump(node_map_dict,f)

Number of nodes after adding category edges:  1472
Number of nodes after adding covisit edges:  2435
Number of nodes after adding geo edges:  2449
Number of nodes after adding visited edges:  17569
Number of nodes after adding friend edges:  20293
Number of nodes after adding category edges:  23763


### Generating Edge List

In [4]:
## Accessing node and edge map dictonary

with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_KG_node_map_dict','rb')as f:
  node_map_dict = pickle.load(f)

with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_KG_edge_map_dict','rb')as f:
  edge_map_dict = pickle.load(f)

In [10]:
len(node_map_dict)

23763

In [11]:
len(edge_map_dict)

7

In [8]:
l1_true = [] #Horizontal
l2_true = [] #Horizontal
l3_true = [] #Vertical

# Adding category edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_category_edges_tf_idf.pkl', 'rb') as f:
    category_edges = pickle.load(f)
for pair in (category_edges):
    head_node = pair[0]
    head_node_id = node_map_dict[head_node]
    tail_node = pair[1]
    tail_node_id = node_map_dict[tail_node]
    relation_id = edge_map_dict["has-category"]
    temp_l = [relation_id]

    l1_true.append(head_node_id)
    l2_true.append(tail_node_id)
    l3_true.append(temp_l)
print("Number of edges after adding category edges: ", len(l1_true))


# Adding covisit edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/poi_covisited_edges.pkl', 'rb') as f:
    covisit_edges = pickle.load(f)
for pair in (covisit_edges):
    head_node = pair[0]
    head_node_id = node_map_dict[head_node]
    tail_node = pair[1]
    tail_node_id = node_map_dict[tail_node]
    relation_id = edge_map_dict["co-visited"]
    temp_l = [relation_id]

    l1_true.append(head_node_id)
    l2_true.append(tail_node_id)
    l3_true.append(temp_l)
print("Number of edges after adding covisit edges: ", len(l1_true))


# Adding geo edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/poi_geocluster_edges.pkl', 'rb') as f:
    geo_edges = pickle.load(f)
for pair in (geo_edges):
    head_node = pair[0]
    head_node_id = node_map_dict[head_node]
    tail_node = pair[1]
    tail_node_id = node_map_dict[tail_node]
    relation_id = edge_map_dict["situated-at"]
    temp_l = [relation_id]

    l1_true.append(head_node_id)
    l2_true.append(tail_node_id)
    l3_true.append(temp_l)
print("Number of edges after adding geo edges: ", len(l1_true))


# Adding visited edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/visited_edges.pkl', 'rb') as f:
    vfive_edges = pickle.load(f)
for pair in (vfive_edges):
    head_node = pair[0]
    head_node_id = node_map_dict[head_node]
    tail_node = pair[1]
    tail_node_id = node_map_dict[tail_node]
    relation_id = edge_map_dict["visited"]
    temp_l = [relation_id]

    l1_true.append(head_node_id)
    l2_true.append(tail_node_id)
    l3_true.append(temp_l)
print("Number of edges after adding visited five edges: ", len(l1_true))


# Adding friend edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/friendship_edges.pkl', 'rb') as f:
    friend_edges = pickle.load(f)
for pair in (friend_edges):
    head_node = pair[0]
    head_node_id = node_map_dict[head_node]
    tail_node = pair[1]
    tail_node_id = node_map_dict[tail_node]
    relation_id = edge_map_dict["friend-with"]
    temp_l = [relation_id]

    l1_true.append(head_node_id)
    l2_true.append(tail_node_id)
    l3_true.append(temp_l)
print("Number of edges after adding friend edges: ", len(l1_true))


# Adding aspect edges
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/shortlisted_poi_significant_aspect_edges.pkl', 'rb') as f:
    aspect_edges = pickle.load(f)
for pair in (aspect_edges):
    head_node = pair[0]
    head_node_id = node_map_dict[head_node]
    tail_node = pair[1]
    tail_node_id = node_map_dict[tail_node]
    relation_id = edge_map_dict["has-aspect"]
    temp_l = [relation_id]

    l1_true.append(head_node_id)
    l2_true.append(tail_node_id)
    l3_true.append(temp_l)
print("Number of edges after adding aspect edges: ", len(l1_true))

Number of edges after adding category edges:  4727
Number of edges after adding covisit edges:  11126
Number of edges after adding geo edges:  12485
Number of edges after adding visited five edges:  166586
Number of edges after adding friend edges:  207163
Number of edges after adding aspect edges:  621263


In [9]:
true_edges = {'edge_index': np.array([l1_true,l2_true]),'edge_reltype': np.array(l3_true)}

## Shuffline the edges for randomisation

combined = list(zip(l1_true, l2_true, l3_true))
random.shuffle(combined)
shuffled_list1, shuffled_list2, shuffled_list3 = zip(*combined)

# Creating split from true edges : This split is for embedding training
# Train set here will hold positive edges for training, negative samples will be created for each of this edge
# Performance of the embeddings will be tested on validation set using conventional embedding loss

train_set_sample_count = int(len(true_edges['edge_reltype'])*0.8)
validation_set_sample_count = int(len(true_edges['edge_reltype'])*0.2)

# Constructing train edges of graph
temp_l1 = []
temp_l2 = []
temp_l3 = []

for i in range(train_set_sample_count):
  temp_l1.append(true_edges['edge_index'][0][i])
  temp_l2.append(true_edges['edge_index'][1][i])
  temp_l3.append([true_edges['edge_reltype'][i][0]])

train_edges = {'edge_index': torch.tensor([temp_l1,temp_l2]),'edge_reltype': torch.tensor(temp_l3)}

# Constructing validation edges of graph
temp_l1 = []
temp_l2 = []
temp_l3 = []

for i in range(train_set_sample_count,len(true_edges['edge_index'][0])):
  temp_l1.append(true_edges['edge_index'][0][i])
  temp_l2.append(true_edges['edge_index'][1][i])
  temp_l3.append([true_edges['edge_reltype'][i][0]])

validation_edges = {'edge_index': torch.tensor([temp_l1,temp_l2]),'edge_reltype': torch.tensor(temp_l3)}

### KG Training

In [15]:
# Hyperparameter Tunning for KG
kg_model = "TransE"
epochs = 100
batch_size = 1000
learning_rate = 1e-3

# Total nodes and relation in dataset
num_entities = len(node_map_dict)
num_relations = len(edge_map_dict)

# Model Initialization
model = TransE(num_entities, num_relations, 50)

In [18]:
# Selecting optimizer for training
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
# Bucketizing data for training
num_workers = os.cpu_count()

# Defining a Relational Dataset object with train set edges and validation set edges
train_dataset = RelationDataset(train_edges, true_edges)
val_dataset = RelationDataset(validation_edges, true_edges)

# Creating dataloader obects as iterators over train and validation Relational Dataset object
# The dataloader internally calls __getitem__ method of object to get the positive and negative triple out of edges
# one bucket from dataloader contains two lists(one for true triple other for false). Each list holds two tensors(one for edges other for relation)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [20]:
# Training Loop

for e in range(epochs):
  losses = []

  model.train()
  for step, batch in enumerate(tqdm.tqdm(train_dataloader, desc="Training")):
    # generate positive as well as negative samples for training
    pos_sample, neg_sample = batch

    # do a forward pass through the model
    entity_embeddings_pass, relation_embeddings_pass = model()

    optimizer.zero_grad()

    # compute the loss as per your model scoring criteria
    loss = TransE_loss(pos_sample[0], neg_sample[0], pos_sample[1], neg_sample[1],
                       entity_embeddings_pass, relation_embeddings_pass)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

  val_losses = []
  model.eval()
  entity_embeddings_pass, relation_embeddings_pass = model()
  # compute validation loss on unseen samples we didn't train on
  for step, batch in enumerate(tqdm.tqdm(val_dataloader, desc="Validating")):
    pos_sample, neg_sample = batch
    loss = TransE_loss(pos_sample[0], neg_sample[0], pos_sample[1], neg_sample[1],
                             entity_embeddings_pass, relation_embeddings_pass)
    val_losses.append(loss.item())

  print(f"epoch: {e + 1} loss: {sum(losses)/len(losses)} val_loss: {sum(val_losses)/len(val_losses)}")

Validating: 100%|██████████| 125/125 [00:10<00:00, 12.07it/s]


epoch: 1 loss: 0.41429636603498077 val_loss: 0.6566273906230926


Validating: 100%|██████████| 125/125 [00:18<00:00,  6.75it/s]


epoch: 2 loss: 0.15857833335138707 val_loss: 0.5684313774108887


Validating: 100%|██████████| 125/125 [00:13<00:00,  9.54it/s]


...

epoch: 98 loss: 0.04473045144424621 val_loss: 0.24968888062238692


Validating: 100%|██████████| 125/125 [00:09<00:00, 12.67it/s]


epoch: 99 loss: 0.04477804722301812 val_loss: 0.25077090376615524


Validating: 100%|██████████| 125/125 [00:10<00:00, 11.62it/s]

epoch: 100 loss: 0.04463488240557981 val_loss: 0.24414815777540208


In [21]:
## Saving the model having trained entity and relation embeddings

with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_KG_model', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Artifact Details

tucson_KG_node_map_dict : {Node_name : Node_index} dictonary of KG

edge_map_dict : {Edge_Type : Edge_Index}dictonary of KG

tucson_KG_model : Trained KG model with nodes and edge embeddings